In [1]:
import numpy as np
import ase
from ase import io
import re
import scipy.spatial.distance
from copy import deepcopy
import ast
import scipy.optimize

In [4]:
class Polymer_Build:
    def __init__(self, path_to_structure):
        
        self.name = str(path_to_structure)
        
        print('Polymer_Build class initialized. Reading and creating ase.Atoms object from {0}'.format(self.name))
        self.path_to_structure = path_to_structure
    
        self.Atoms = ase.io.read(self.path_to_structure)
        with open(self.path_to_structure) as f:
            comments = re.search('%%%(.*)%%%', f.readlines()[1]).group(1)
        f.close()
        
        self.attachment_pts_dict = ast.literal_eval(comments) # this is unsafe/unsecure blah blah blah
        print('Attachment points on this molecule are: {0}\n\n'.format(str(self.attachment_pts_dict)))
        
        self.LHS_Reflection_Needed = None
        self.RHS_Reflection_Needed = None

        
    def add_FG(self, FG_Polymer_Build_Object, location=['LHS', 'RHS']):
        # there is a distinction between "global" coordinates, which refer to a position in the Polymer_Build Ensemble (i.e. after >=1 FG has been added) and "local" coordinates, which are positions in the xyz file of the isolated FG being added
        if location == 'LHS':
            
            print('Adding {0} to the {1} of structure {2}'.format(FG_Polymer_Build_Object.name, location, self.name))
            
            added_obj = deepcopy(FG_Polymer_Build_Object)
            
            for key, value in self.attachment_pts_dict.items():
                # find the LHS attachment pt on the target molecule (self)
                if 'L' in key:
                    
                    # calculate the Euclidean distances between the LHS attachment site (on target molecule) and every other atom
                    backbone_LHS_distances_tmp = [] # will take the form [Euclidean distance, chemical symbol, global coordinates (relative to target molecule)]
                    for index, value in enumerate(self.Atoms.get_positions()):
                        backbone_LHS_distances_tmp.append(
                            [np.linalg.norm(value-self.Atoms.get_positions()[self.attachment_pts_dict['L']['index']]),
                             self.Atoms.get_chemical_symbols()[index],
                             value]
                        )
                    
                    # filter SORTED list
                    backbone_LHS_distances_tmp = sorted(backbone_LHS_distances_tmp,key=lambda x: (x[0]))
                    hidden = backbone_LHS_distances_tmp.pop(0) # first element will always be 0 because when i=j
                
                    if backbone_LHS_distances_tmp[0][1] == 'C' and backbone_LHS_distances_tmp[1][1] == 'C':
                        # case where closest 2 atoms to backbone LHS are both C
                        print('Nearest atoms to backbone LHS are both C.')
                        LHS_nearest_atom = backbone_LHS_distances_tmp[0]
                        print('The nearest atom to backbone LHS is {0}'.format(LHS_nearest_atom))
                        self.LHS_Reflection_Needed = False
                    else:
                        print('Nearest atoms to backbone LHS are NOT both C. Filtering NNs list')
                        backbone_LHS_distances_tmp = filter(lambda c: c[1]!='C', sorted(backbone_LHS_distances_tmp,key=lambda x: (x[0])))
                        backbone_LHS_distances_tmp = list(filter(lambda c: c[1]!='H', backbone_LHS_distances_tmp))
                        print('Nearest non-C atom to backbone LHS is: {0}'.format(backbone_LHS_distances_tmp[0]))
                        LHS_nearest_atom = backbone_LHS_distances_tmp[0]
                        self.LHS_Reflection_Needed = True
                    

                    
                    # drawing a vector from LHS_nearest_atom to LHS attachment pt, scaling slightly, then adding to coords of LHS attachment pt
                    translation_vector = self.Atoms.get_positions()[self.attachment_pts_dict['L']['index']] - np.array([1.45, 0, 0])#((self.Atoms.get_positions()[self.attachment_pts_dict['L']['index']] - LHS_nearest_atom[2]) * C_scaling) + self.Atoms.get_positions()[self.attachment_pts_dict['L']['index']]

                    
                    print('Placing a C atom at {0} in global coordinates'.format(translation_vector))
                    
                    # have vector between nearest non C and terminal C, need to put C on the end of it
                    self.Atoms.append(ase.Atom('C', translation_vector))
                    self.attachment_pts_dict['L'] = {'index': -1, 'type': 'FG'}
                    
                    # set positions of added object
                    print('Target structure attachment points: {0}'.format(self.attachment_pts_dict))
                    
                    # calculate the Euclidean distances between the FG RHS attachment site and every other atom
                    FG_RHS_distances_tmp = [] # format: [Euclidean distance, chemical symbol, index, local coordinates]
                    for index, value in enumerate(FG_Polymer_Build_Object.Atoms.get_positions()):
                        FG_RHS_distances_tmp.append(
                            [np.linalg.norm(value-FG_Polymer_Build_Object.Atoms.get_positions()[FG_Polymer_Build_Object.attachment_pts_dict['R']['index']]),
                             FG_Polymer_Build_Object.Atoms.get_chemical_symbols()[index],
                             index,
                             value]
                        )
                    
                    # filter sorted list
                    FG_RHS_distances_tmp = sorted(FG_RHS_distances_tmp,key=lambda x: (x[0]))
                    hidden = FG_RHS_distances_tmp.pop(0) # the first distance will be the identity
                    FG_RHS_distances_tmp_before_filter = deepcopy(FG_RHS_distances_tmp)
                    FG_RHS_distances_tmp = filter(lambda c: c[1]!='C', FG_RHS_distances_tmp)
                    FG_RHS_distances_tmp = list(filter(lambda c: c[1]!='H', FG_RHS_distances_tmp))
                
                    if not FG_RHS_distances_tmp:
                        print('The FG you are adding has only C and H.')
                        FG_Polymer_Build_Object.RHS_Reflection_Needed = False
                        RHS_nearest_atom = FG_RHS_distances_tmp_before_filter[0]
                        print('The nearest atom is {0}'.format(RHS_nearest_atom))
                    elif FG_RHS_distances_tmp[0][0] < 5:
                        RHS_nearest_atom = list(FG_RHS_distances_tmp[0])
                        print('before addition, the closest non-H-C atom on FG RHS C is')
                        print(RHS_nearest_atom)
                        FG_Polymer_Build_Object.RHS_Reflection_Needed = True
                        
                    ##### at this point, we have identified the nearest atoms to the attachment points on the target molecule and the FG, and defined conditions for if reflection about conjugation axis is necessary

                    # test intial configuration, then test if mirrored FG configuration maximizes C-to-nearest distance

                    # R to L vector of FG
                    FG_R_to_L_vector = FG_Polymer_Build_Object.Atoms.get_positions()[FG_Polymer_Build_Object.attachment_pts_dict['L']['index']] - FG_Polymer_Build_Object.Atoms.get_positions()[FG_Polymer_Build_Object.attachment_pts_dict['R']['index']]

                    print('FG R TO L VECTOR: {0}'.format(FG_R_to_L_vector))

                    # this vector vectorizes all positions relative to the RHS attachment point
                    FG_R_to_all_atoms_vectors = FG_Polymer_Build_Object.Atoms.get_positions() - FG_Polymer_Build_Object.Atoms.get_positions()[FG_Polymer_Build_Object.attachment_pts_dict['R']['index']]

                    # rotation matrix created from here: https://en.wikipedia.org/wiki/Rotation_matrix#Rotation_matrix_from_axis_and_angle
                    theta = np.deg2rad(179)
                    
                    (x, y, z) = FG_R_to_L_vector / np.linalg.norm(FG_R_to_L_vector)

                    def generate_rotation_matrix(theta, x, y, z):
                        
                        c = np.cos(theta)
                        s = np.sin(theta)
                        C = 1 - c
                        
                        return np.array([
                            [x*x*C+c, x*y*C-z*s, x*z*C+y*s],
                            [y*x*C+z*s, y*y*C+c, y*z*C-z*s],
                            [z*x*C-y*s, z*y*C+x*s, z*z*C+c]
                        ])

                    C_link_position = self.Atoms.get_positions()[-1]
                    trial_coords_2_rotated_vectors = FG_R_to_all_atoms_vectors.dot(generate_rotation_matrix(theta, x, y, z))
                
                    if self.LHS_Reflection_Needed and FG_Polymer_Build_Object.RHS_Reflection_Needed:
                    
                        print('Both LHS and RHS reflection needed.')
                        
                        # find the distance between nearest atom on FG with RHS @ newly placed C and LHS nearest atom
                        distance_1 = np.linalg.norm((self.Atoms.get_positions()[-1] + FG_R_to_all_atoms_vectors[RHS_nearest_atom[2]]) - (LHS_nearest_atom[2])
                                                   )
                        distance_2 = np.linalg.norm((self.Atoms.get_positions()[-1] + trial_coords_2_rotated_vectors[RHS_nearest_atom[2]]) - (LHS_nearest_atom[2])
                                                   )
                        
                        self.Atoms.pop(-1)
                        
                        if distance_2 > distance_1:
                            # reflected configuration is optimal
                            added_obj.Atoms.set_positions(trial_coords_2_rotated_vectors + C_link_position) #############
                            added_obj.attachment_pts_dict['L']['index'] = added_obj.attachment_pts_dict['L']['index'] + len(self.Atoms.get_positions())
                            self.attachment_pts_dict['L'] = added_obj.attachment_pts_dict['L']

                            for index, value in enumerate(added_obj.Atoms):
                                self.Atoms.append(value)

                            print('Added requested FG to LHS of target molecule in reflected configuration. New attachment points on target are now: {0}\n\n'.format(self.attachment_pts_dict))
                            # is self.last_added_LHS deprecated?
                    
                        else:
                            # default configuration is optimal
                            added_obj.Atoms.set_positions(FG_R_to_all_atoms_vectors + C_link_position) ################
                            added_obj.attachment_pts_dict['L']['index'] = added_obj.attachment_pts_dict['L']['index'] + len(self.Atoms.get_positions())
                            self.attachment_pts_dict['L'] = added_obj.attachment_pts_dict['L']

                            for index, value in enumerate(added_obj.Atoms):
                                self.Atoms.append(value)

                            print('Added requested FG to LHS of target molecule. New attachment points on target are now: {0}\n\n'.format(self.attachment_pts_dict))
         
                    else:
                        self.Atoms.pop(-1)
                        added_obj.Atoms.set_positions(FG_R_to_all_atoms_vectors + C_link_position) ###################
                        added_obj.attachment_pts_dict['L']['index'] = added_obj.attachment_pts_dict['L']['index'] + len(self.Atoms.get_positions())
                        self.attachment_pts_dict['L'] = added_obj.attachment_pts_dict['L']

                        for index, value in enumerate(added_obj.Atoms):
                            self.Atoms.append(value)

                        print('Added requested FG to LHS of target molecule. New attachment points on target are now: {0}\n\n'.format(self.attachment_pts_dict))
                        
        if location == 'RHS':
            
            print('Adding {0} to the {1} of structure {2}'.format(FG_Polymer_Build_Object.name, location, self.name))
            
            added_obj = deepcopy(FG_Polymer_Build_Object)
            
            for key, value in self.attachment_pts_dict.items():
                # find the LHS attachment pt on the target molecule (self)
                if 'R' in key:
                    
                    # calculate the Euclidean distances between the LHS attachment site (on target molecule) and every other atom
                    backbone_RHS_distances_tmp = [] # will take the form [Euclidean distance, chemical symbol, global coordinates (relative to target molecule)]
                    for index, value in enumerate(self.Atoms.get_positions()):
                        backbone_RHS_distances_tmp.append(
                            [np.linalg.norm(value-self.Atoms.get_positions()[self.attachment_pts_dict['R']['index']]),
                             self.Atoms.get_chemical_symbols()[index],
                             value]
                        )
                    
                    # filter SORTED list
                    backbone_RHS_distances_tmp = sorted(backbone_RHS_distances_tmp,key=lambda x: (x[0]))
                    hidden = backbone_RHS_distances_tmp.pop(0) # first element will always be 0 because when i=j
                
                    if backbone_RHS_distances_tmp[0][1] == 'C' and backbone_RHS_distances_tmp[1][1] == 'C':
                        # case where closest 2 atoms to backbone RHS are both C
                        print('Nearest atoms to backbone RHS are both C.')
                        RHS_nearest_atom = backbone_RHS_distances_tmp[0]
                        print('The nearest atom to backbone RHS is {0}'.format(RHS_nearest_atom))
                        self.RHS_Reflection_Needed = False
                    else:
                        print('Nearest atoms to backbone RHS are NOT both C. Filtering NNs list')
                        backbone_RHS_distances_tmp = filter(lambda c: c[1]!='C', sorted(backbone_RHS_distances_tmp,key=lambda x: (x[0])))
                        backbone_RHS_distances_tmp = list(filter(lambda c: c[1]!='H', backbone_RHS_distances_tmp))
                        print('Nearest non-C atom to backbone RHS is: {0}'.format(backbone_RHS_distances_tmp[0]))
                        RHS_nearest_atom = backbone_RHS_distances_tmp[0]
                        self.RHS_Reflection_Needed = True
                    

                    
                    # drawing a vector from RHS_nearest_atom to RHS attachment pt, scaling slightly, then adding to coords of RHS attachment pt
                    translation_vector = self.Atoms.get_positions()[self.attachment_pts_dict['R']['index']] + np.array([1.45, 0, 0])

                    
                    print('Placing a C atom at {0} in global coordinates'.format(translation_vector))
                    
                    # have vector between nearest non C and terminal C, need to put C on the end of it
                    self.Atoms.append(ase.Atom('C', translation_vector))
                    self.attachment_pts_dict['R'] = {'index': -1, 'type': 'FG'}
                    
                    # set positions of added object
                    print('Target structure attachment points: {0}'.format(self.attachment_pts_dict))
                    
                    # calculate the Euclidean distances between the FG RHS attachment site and every other atom
                    FG_LHS_distances_tmp = [] # format: [Euclidean distance, chemical symbol, index, local coordinates]
                    for index, value in enumerate(FG_Polymer_Build_Object.Atoms.get_positions()):
                        FG_LHS_distances_tmp.append(
                            [np.linalg.norm(value-FG_Polymer_Build_Object.Atoms.get_positions()[FG_Polymer_Build_Object.attachment_pts_dict['L']['index']]),
                             FG_Polymer_Build_Object.Atoms.get_chemical_symbols()[index],
                             index,
                             value]
                        )
                    
                    # filter sorted list
                    FG_LHS_distances_tmp = sorted(FG_LHS_distances_tmp,key=lambda x: (x[0]))
                    hidden = FG_LHS_distances_tmp.pop(0) # the first distance will be the identity
                    FG_LHS_distances_tmp_before_filter = deepcopy(FG_LHS_distances_tmp)
                    FG_LHS_distances_tmp = filter(lambda c: c[1]!='C', FG_LHS_distances_tmp)
                    FG_LHS_distances_tmp = list(filter(lambda c: c[1]!='H', FG_LHS_distances_tmp))
                
                    if not FG_LHS_distances_tmp:
                        print('The FG you are adding has only C and H.')
                        FG_Polymer_Build_Object.LHS_Reflection_Needed = False
                        LHS_nearest_atom = FG_LHS_distances_tmp_before_filter[0]
                        print('The nearest atom is {0}'.format(LHS_nearest_atom))
                    elif FG_LHS_distances_tmp[0][0] < 5:
                        LHS_nearest_atom = list(FG_LHS_distances_tmp[0])
                        print('before addition, the closest non-H-C atom on FG RHS C is')
                        print(LHS_nearest_atom)
                        FG_Polymer_Build_Object.LHS_Reflection_Needed = True
                        
                    ##### at this point, we have identified the nearest atoms to the attachment points on the target molecule and the FG, and defined conditions for if reflection about conjugation axis is necessary

                    # test intial configuration, then test if 180-deg rotated FG configuration maximizes C-to-nearest distance

                    # L to R vector of FG
                    FG_L_to_R_vector = FG_Polymer_Build_Object.Atoms.get_positions()[FG_Polymer_Build_Object.attachment_pts_dict['R']['index']] - FG_Polymer_Build_Object.Atoms.get_positions()[FG_Polymer_Build_Object.attachment_pts_dict['L']['index']]

                    print('FG L TO R VECTOR: {0}'.format(FG_L_to_R_vector))

                    # this vector vectorizes all positions relative to the RHS attachment point
                    FG_L_to_all_atoms_vectors = FG_Polymer_Build_Object.Atoms.get_positions() - FG_Polymer_Build_Object.Atoms.get_positions()[FG_Polymer_Build_Object.attachment_pts_dict['L']['index']]

                    # rotation matrix created from here: https://en.wikipedia.org/wiki/Rotation_matrix#Rotation_matrix_from_axis_and_angle
                    theta = np.deg2rad(179)
                    
                    (x, y, z) = FG_L_to_R_vector / np.linalg.norm(FG_L_to_R_vector)

                    def generate_rotation_matrix(theta, x, y, z):
                        
                        c = np.cos(theta)
                        s = np.sin(theta)
                        C = 1 - c
                        
                        return np.array([
                            [x*x*C+c, x*y*C-z*s, x*z*C+y*s],
                            [y*x*C+z*s, y*y*C+c, y*z*C-z*s],
                            [z*x*C-y*s, z*y*C+x*s, z*z*C+c]
                        ])

                    C_link_position = self.Atoms.get_positions()[-1]
                    trial_coords_2_rotated_vectors = FG_L_to_all_atoms_vectors.dot(generate_rotation_matrix(theta, x, y, z))
                
                    if self.RHS_Reflection_Needed and FG_Polymer_Build_Object.LHS_Reflection_Needed:
                    
                        print('Both RHS and LHS reflection needed.')
                        
                        # find the distance between nearest atom on FG with RHS @ newly placed C and LHS nearest atom
                        distance_1 = np.linalg.norm((self.Atoms.get_positions()[-1] + FG_L_to_all_atoms_vectors[LHS_nearest_atom[2]]) - (RHS_nearest_atom[2])
                                                   )
                        distance_2 = np.linalg.norm((self.Atoms.get_positions()[-1] + trial_coords_2_rotated_vectors[LHS_nearest_atom[2]]) - (RHS_nearest_atom[2])
                                                   )
                        
                        self.Atoms.pop(-1)
                        
                        if distance_2 > distance_1:
                            # reflected configuration is optimal
                            added_obj.Atoms.set_positions(trial_coords_2_rotated_vectors + C_link_position) #############
                            added_obj.attachment_pts_dict['R']['index'] = added_obj.attachment_pts_dict['R']['index'] + len(self.Atoms.get_positions())
                            self.attachment_pts_dict['R'] = added_obj.attachment_pts_dict['R']

                            for index, value in enumerate(added_obj.Atoms):
                                self.Atoms.append(value)

                            print('Added requested FG to RHS of target molecule in reflected configuration. New attachment points on target are now: {0}\n\n'.format(self.attachment_pts_dict))
                    
                        else:
                            # default configuration is optimal
                            added_obj.Atoms.set_positions(FG_L_to_all_atoms_vectors + C_link_position) ################
                            added_obj.attachment_pts_dict['R']['index'] = added_obj.attachment_pts_dict['R']['index'] + len(self.Atoms.get_positions())
                            self.attachment_pts_dict['R'] = added_obj.attachment_pts_dict['R']

                            for index, value in enumerate(added_obj.Atoms):
                                self.Atoms.append(value)

                            print('Added requested FG to RHS of target molecule. New attachment points on target are now: {0}\n\n'.format(self.attachment_pts_dict))
         
                    else:
                        self.Atoms.pop(-1)
                        added_obj.Atoms.set_positions(FG_L_to_all_atoms_vectors + C_link_position) ###################
                        added_obj.attachment_pts_dict['R']['index'] = added_obj.attachment_pts_dict['R']['index'] + len(self.Atoms.get_positions())
                        self.attachment_pts_dict['R'] = added_obj.attachment_pts_dict['R']

                        for index, value in enumerate(added_obj.Atoms):
                            self.Atoms.append(value)

                        print('Added requested FG to RHS of target molecule. New attachment points on target are now: {0}\n\n'.format(self.attachment_pts_dict))
            
            

In [ ]:
def terminate('add methyl to *HS')

In [6]:
a = Polymer_Build('DPP-template.xyz')
a.name = 'backbone'
# a.attachment_pts_dict
b = Polymer_Build('A/T.xyz')
c = Polymer_Build('A/benzene.xyz')
d = Polymer_Build('A/EDOT.xyz')
a.add_FG(b, 'RHS')
a.add_FG(b, 'RHS')
a.add_FG(c, 'RHS')
a.add_FG(c, 'RHS')
a.add_FG(d, 'RHS')
a.add_FG(d, 'RHS')
a.add_FG(b, 'LHS')
a.add_FG(b, 'LHS')
a.add_FG(c, 'LHS')
a.add_FG(c, 'LHS')
a.add_FG(d, 'LHS')
a.add_FG(d, 'LHS')
# a.add_FG(b, 'RHS')
a.Atoms.write('test-LR.xyz')
# print(a.attachment_pts_dict)

Polymer_Build class initialized. Reading and creating ase.Atoms object from DPP-template.xyz
Attachment points on this molecule are: {'L': {'index': 1, 'type': 'FG'}, 'R': {'index': 3, 'type': 'FG'}, 'S1': {'index': 0, 'type': 'AL'}, 'S2': {'index': 2, 'type': 'AL'}}


Polymer_Build class initialized. Reading and creating ase.Atoms object from A/T.xyz
Attachment points on this molecule are: {'L': {'index': 0, 'type': 'FG'}, 'R': {'index': 1, 'type': 'FG'}}


Polymer_Build class initialized. Reading and creating ase.Atoms object from A/benzene.xyz
Attachment points on this molecule are: {'L': {'index': 0, 'type': 'FG'}, 'R': {'index': 1, 'type': 'FG'}}


Polymer_Build class initialized. Reading and creating ase.Atoms object from A/EDOT.xyz
Attachment points on this molecule are: {'L': {'index': 0, 'type': 'FG'}, 'R': {'index': 1, 'type': 'FG'}}


Adding A/T.xyz to the RHS of structure backbone
Nearest atoms to backbone RHS are NOT both C. Filtering NNs list
Nearest non-C atom to backbon

In [161]:
def distance(x1, x2):
    return np.linalg.norm(x1-x2)

distance(np.array([0, 0, 0]), np.array([1, 1, 1]))

1.7320508075688772